API em URL Público
Criação de uma conta no site da https://ngrok.com/

Pós-criação, copiar a API_KEY do link: https://dashboard.ngrok.com/get-started/your-authtoken

Criar um dominio no Ngrok https://dashboard.ngrok.com/cloud-edge/domains

Variáveis de ambiente
É necessário criar duas secrets aqui no colab (Ícone de chave na lateral direita do menu)

As chaves necessárias são:

NGROK_TOKEN
NGROK_DOMAIN

In [ ]:
from google.colab import userdata

# Token de acesso ao Ngrok
NGROK_API_KEY = userdata.get('NGROK_TOKEN')

# Dominio criado no Ngrok
NGROK_DOMAIN = userdata.get('NGROK_DOMAIN')

# Escolha do modelo Ollama para uso
MODEL_NAME_OLLAMA = 'mistral'

Instalação de Ollama no Google Colab Server

In [ ]:
# Download and install ollama to the system
# Instale os pré-requisitos no Colab
!apt-get update && apt-get install -y curl sudo

# Baixe o script e modifique para evitar erros relacionados ao systemd
!curl https://ollama.ai/install.sh -o install.sh
!sed -i 's/Creating ollama systemd service.*//g' install.sh
!sed -i 's/systemctl.*//g' install.sh

# Execute o script modificado
!bash install.sh


Instalação de pacotes Python

In [ ]:
# execução e criação da API
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

os.environ['OLLAMA_HOST'] = '0.0.0.0'


async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

#register an account at ngrok.com and create an authtoken and place it here
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', NGROK_API_KEY])
)

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'run', MODEL_NAME_OLLAMA]),
    run_process(['ngrok', 'http', f'--domain={NGROK_DOMAIN}', '--log', 'stderr', '11434']),
)

In [ ]:
!pip install -qU aiohttp pyngrok